In [1]:
import pandas as pd
import ast

data_path = '../paired_data.csv'

data = pd.read_csv(data_path)
data["overview_embedding"] = data["overview_embedding"].apply(ast.literal_eval)

In [2]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

/home/matt/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/matt/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from joblib import dump

# Convert embedding column from list to numpy array
data["overview_embedding"] = data["overview_embedding"].apply(np.array)

# Split the data into features (X) and target (y)
X = np.vstack(data["overview_embedding"].values)
y = data['average_rating']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define and train the linear regression model
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

# Save the scaler and the model
dump(scaler, 'scaler.joblib')
dump(model, 'movie_rating_prediction_model.joblib')

Mean Squared Error: 0.48981401284178977


['movie_rating_prediction_model.joblib']

In [17]:
import pandas as pd
from sentence_transformers import SentenceTransformer

# Load the SentenceTransformer model
transformer = SentenceTransformer('all-MiniLM-L6-v2')

overview: str = input("Give your movie overview: ")
print(overview)

# Now you can use the loaded scaler to transform new data
scaled_embed = scaler.transform(np.array(transformer.encode(overview)).reshape(1, -1))

predicted_rating = model.predict(scaled_embed)

print(f"prediction: {predicted_rating}")


/home/matt/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


An urbane fox cannot resist returning to his farm raiding ways and then must help his community survive the farmers' retaliation.
prediction: [2.97658728]
